# Neural Network from sklearn
Conny Lin | June 15, 2020 | updated June 18, 2020

[sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)


In [1]:
# import local libraries using host specific paths
import socket, sys
hostname = socket.gethostname().split('.')[0]
# set local path settings based on computer host
if hostname == 'PFC':
    pylibrary = '/Users/connylin/Dropbox/Code/proj'
elif hostname == 'Angular-Gyrus':
    pylibrary = '/Users/connylin/Code/proj'
else:
    assert False, 'host computer not regonized'
# load local libraries
if pylibrary not in sys.path:
    sys.path.insert(1, pylibrary)
# import brainstation_capstone
# import other standard paths and local variables
from brainstation_capstone.system import host_paths
localpaths = host_paths.get(hostname)
datapath = localpaths['datapath']

# import standard libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt# import local libraries using host specific paths

getting host computer specific paths


Try a simple NN from sklearn. But usually won't use this package. Normally use packages like *Keras*, *TensorFlow*, *PyTorch*, etc., which are all specifically designed for neural networks.

In [8]:
def PlotBoundaries(model, X, Y) :
    '''
    Helper function that plots the decision boundaries of a model and data (X,Y)
    code modified from: https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html
    '''
    
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1,X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.4)

    #Plot
    plt.scatter(X[:, 0], X[:, 1], c=Y,s=20, edgecolor='k')
    plt.show()

## Try simple NN from sklearn

In [7]:
# try simple NN
from sklearn.neural_network import MLPClassifier
# import data
from brainstation_capstone.etl.datatransform import Nutcracker
X, X_test, y, y_test = Nutcracker(datapath).mldata(feature_reduction='None')

None
None


In [13]:
NN_model = MLPClassifier(hidden_layer_sizes=(1),solver='lbfgs')
NN_model.fit(X_train, y_train);

In [15]:
print(f"Train Score: {NN_model.score(X_train,y_train): 0.3f}")
print(f"Test Score: {NN_model.score(X_test,y_test): 0.3f}")

Train Score:  0.869
Test Score:  0.869


solver lbfgs and layer size =1 gives similar results as other ML models.

In [13]:
import timeit
from sklearn.neural_network import MLPClassifier
# import data
from brainstation_capstone.etl.datatransform import Nutcracker
X, X_test, y, y_test = Nutcracker(datapath).mldata(feature_reduction='None')
# set up time
start_time = timeit.timeit()
print('timer start')
print(f'test timer {(timeit.timeit() - start_time)/60:.2f} min')
# start model
NN_model = MLPClassifier()
NN_model.fit(X, y)
# print results
print(f"Train Score: {NN_model.score(X, y): 0.3f}")
print(f"Test Score: {NN_model.score(X_test, y_test): 0.3f}")
# print time elapsed
print(f'took {(timeit.timeit() - start_time)/60:.2f} min')

counter start
test timer 0.00 min
Train Score:  0.913
Test Score:  0.913
took -0.00 min


untuned model gives 91% results. highest of all ML models

find out what hyperparamters the untuned model used.

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

* hidden_layer_sizes: 100
* activation: relu
    * {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default=’relu’
    * try different activation
* solver: ’adam’
* alpha: 0.0001
    * higher more regularlized. (less over fit)
    * test and train score are similar. doesn't seem like too much overfit.
* batch_size: ’auto’
* learning_rate: constant
* learning_rate_init: 0.001
* power_t: 0.5
* max_iter: 200
* shuffle: True
* random_state: None
* tol: 1e-4
* verbose: False
* warm_start: False
* momentum: 0.9
* nesterov_momentum: True
* early_stopping: False
* validation_fraction: 0.1
* beta_1: 0.9
* beta_2: 0.999
* epsilon: 1e-8
* n_iter_no_change: 10
* max_fun: 15000

## Try different activation method | June 19, 09:43

In [2]:
# BUG: this code has bug on timeit and test scores not being stored
import timeit
from sklearn.neural_network import MLPClassifier
# import data
from brainstation_capstone.etl.datatransform import Nutcracker
X, X_test, y, y_test = Nutcracker(datapath).mldata(feature_reduction='None')
# set up time
start_time_all = timeit.timeit()
print('timer start')
print(f'test timer {(timeit.timeit() - start_time_all)/60:.2f} min')
# hyperparameters
activation_choice = ['identity', 'logistic', 'tanh', 'relu']
# declare output var
test_scores = []
train_scores = []
# test hyperparameter
for activation in activation_choice:
    print(f'running {activation}')
    # set up time
    start_time = timeit.timeit()
    # start model
    NN_model = MLPClassifier(activation=activation)
    NN_model.fit(X, y)
    # get scores
    train_score = NN_model.score(X, y)
    test_score = NN_model.score(X_test, y_test)
    # print results
    print(f"\tTrain Score: {train_score}")
    print(f"\tTest Score: {test_score}")
    # print time elapsed
    print(f'\ttook {(timeit.timeit() - start_time)/60:.2f} min')

# set up time
start_time_all = timeit.timeit()
print(f'total time: {(timeit.timeit() - start_time_all)/60:.2f} min')
                        

None
None
timer start
test timer -0.00 min
running identity
	Train Score: 0.869185
	Test Score: 0.868695
	took 0.00 min
running logistic
	Train Score: 0.917125625
	Test Score: 0.91604
	took -0.00 min
running tanh
	Train Score: 0.91812375
	Test Score: 0.91745
	took 0.00 min
running relu
	Train Score: 0.91652875
	Test Score: 0.915915
	took -0.00 min
total time: 0.00 min


In [5]:
range(len(activation_choice))

range(0, 4)

In [4]:
# graph results
from brainstation_capstone.vs.plots import hyperparameterplot
hyperparameterplot(range(len(activation_choice)), train_scores, test_scores, 'activation', 'NN sklearn')

ValueError: x and y must have same first dimension, but have shapes (4,) and (0,)

Try TensorFlow. This must be run in the deeplearning env.